In [ ]:
import sys
import threading
import dash
from dash import dcc, html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from PyQt6.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget
from PyQt6.QtWebEngineWidgets import QWebEngineView
import time

# Sample data
points = ['A', 'B', 'C']
x = [1, 2, 3]
y = [3, 1, 2]
images = {
    "A": "https://cdn0.ecologiaverde.com/es/posts/0/5/6/sector_agricola_que_es_actividades_e_importancia_4650_600.webp",
    "B": "https://cdn0.ecologiaverde.com/es/posts/0/5/6/que_es_el_sector_agricola_4650_0_600.webp",
    "C": "https://cdn0.ecologiaverde.com/es/posts/0/5/6/sector_agricola_que_es_actividades_e_importancia_4650_600.webp"
}

# Initialize Dash app
app = dash.Dash(__name__)

# Create scatter plot
fig = go.Figure(go.Scatter(
    x=x, y=y, mode="markers",
    marker=dict(size=10),
    hoverinfo="text",
    text=points  # Point labels
))

app.layout = html.Div([
    # Image container (above the scatter plot)
    html.Div([
        html.Img(id="hover-image", src="", style={
            "width": "200px",
            "display": "none",
            "margin-bottom": "10px"
        })
    ], style={"display": "flex", "justify-content": "center", "align-items": "center", "height": "220px"}),

    # Scatter plot
    dcc.Graph(id="scatter-plot", figure=fig)
])


# Callback to update image on hover
@app.callback(
    Output("hover-image", "src"),
    Output("hover-image", "style"),
    Input("scatter-plot", "hoverData")
)
def display_hover_image(hoverData):
    if hoverData:
        point_name = hoverData["points"][0]["text"]
        img_src = images.get(point_name, "")
        return img_src, {
            "width": "200px",
            "display": "block"
        } if img_src else {"display": "none"}
    return "", {"display": "none"}


# Function to run Dash app in a separate thread
def run_dash():
    app.run_server(debug=False, port=8050, use_reloader=False)


# PyQt6 Application
class DashViewer(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Plotly Dash in PyQt6")
        self.setGeometry(100, 100, 1000, 700)

        # Create QWebEngineView widget
        self.browser = QWebEngineView()

        # Set layout
        central_widget = QWidget()
        layout = QVBoxLayout()
        layout.addWidget(self.browser)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Start Dash app in background thread
        self.start_dash_server()

        # Load the Dash app in QWebEngineView
        time.sleep(2)  # Wait for the server to start
        self.browser.setUrl("http://127.0.0.1:8050")

    def start_dash_server(self):
        dash_thread = threading.Thread(target=run_dash, daemon=True)
        dash_thread.start()


# Run PyQt application
if __name__ == "__main__":
    app_qt = QApplication(sys.argv)
    viewer = DashViewer()
    viewer.show()
    sys.exit(app_qt.exec())
